In [3]:
import pandas as pd
import io
import matplotlib.pyplot as plt
import os
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, Table, TableStyle
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

# 파일 이름 (사용자 업로드 파일)
CSV_FILE_NAME = "예금은행_대출금리_신규취급액_기준__20251027223849.csv"

# ReportLab 한글 폰트 설정 (오류 방지 로직 포함)
HANGEUL_FONT = 'Helvetica-Bold'
try:
    # NanumGothicBold.ttf 파일 경로가 필요합니다. 없으면 기본 폰트 사용
    pdfmetrics.registerFont(TTFont('NanumGothic', 'NanumGothicBold.ttf'))
    HANGEUL_FONT = 'NanumGothic'
except Exception:
    print("WARNING: ReportLab 한글 폰트 등록 실패. PDF 내 한글이 깨질 수 있습니다.")

# ===============================================
# 1. DATA_PREP: 데이터 로딩 및 정리 (4단계 시작)
# ===============================================

def load_and_preprocess_data(csv_file_name):
    """CSV 파일을 로드하고 시계열 데이터로 정제합니다."""
    
    # [3단계 결과물] 텍스트 파일 내용을 읽는 도구 호출
    file_content = content_fetcher.fetch(file_name=csv_file_name)
    csv_data = file_content['fullText']

    # CSV 데이터를 Pandas DataFrame으로 변환 (첫 행을 헤더로, 첫 컬럼을 인덱스로)
    df_raw = pd.read_csv(io.StringIO(csv_data), index_col=0, encoding='utf-8')
    
    # 인덱스(항목)와 컬럼(시간)을 바꿔 시계열 데이터(시간 축)로 만듭니다.
    df_processed = df_raw.T
    
    # 인덱스(시간)를 날짜 타입으로 변환
    df_processed.index = pd.to_datetime(df_processed.index, format='%Y.%m')

    # 시각화할 핵심 컬럼 정의
    loan_cols = ['대출평균 1) (연%)', '가계대출 (연리%)', '주택담보대출 (연리%)']
    
    # 금리 데이터(문자열)를 숫자(float)로 변환
    for col in loan_cols:
        if col in df_processed.columns:
            df_processed[col] = df_processed[col].astype(str).str.replace(',', '', regex=False).astype(float)
        else:
            print(f"경고: 컬럼 '{col}'이 데이터에 없습니다.")
            
    print("--- 4단계: 데이터 로딩 및 정리 완료 ---")
    print(df_processed[loan_cols].tail())
    
    return df_processed[loan_cols]

# ===============================================
# 2. DRAW_CHARTS: 선 그래프 생성 및 저장 (4단계 지속)
# ===============================================

def generate_line_chart(df_data, chart_dir):
    """정제된 데이터로 선 그래프를 그리고 파일로 저장합니다."""
    
    os.makedirs(chart_dir, exist_ok=True)
    image_filename = os.path.join(chart_dir, '대출금리_추이_선그래프.png')
    
    plt.figure(figsize=(10, 6))
    
    # Matplotlib 한글 폰트 설정 (Windows 기본 폰트 사용)
    try:
        plt.rcParams['font.family'] = 'Malgun Gothic'
    except Exception:
        print("WARNING: Matplotlib 한글 폰트 설정 실패. 그래프 제목이 깨질 수 있습니다.")
    
    # 선 그래프 그리기 (시간에 따른 변화를 보여줍니다.)
    for col in df_data.columns:
        plt.plot(df_data.index, df_data[col], marker='o', linestyle='-', label=col.split('(')[0].strip())

    # 차트 꾸미기
    plt.title('예금은행 신규취급액 기준 대출금리 추이', fontsize=16)
    plt.xlabel('월', fontsize=12)
    plt.ylabel('금리 (연 %)', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend(fontsize=10)
    plt.tight_layout()
    
    # 그림 파일 저장
    plt.savefig(image_filename)
    plt.close()
    
    print("\n--- 4단계: 시각화 자료 (선 그래프) 생성 및 저장 완료 ---")
    print(f"저장된 파일: {image_filename}")
    return image_filename

# ===============================================
# 3. FINAL_REPORT: PDF 보고서 완성 (5단계 시작)
# ===============================================

def create_pdf_report(df_data, image_path, report_dir):
    """표와 그림을 PDF 파일로 통합합니다."""
    
    os.makedirs(report_dir, exist_ok=True)
    pdf_filename = os.path.join(report_dir, "은행_대출금리_추이_보고서.pdf")
    
    doc = SimpleDocTemplate(pdf_filename, pagesize=A4)
    styles = getSampleStyleSheet()
    
    # ReportLab 스타일에서 한글 폰트 설정 적용
    styles['Title'].fontName = HANGEUL_FONT
    styles['h2'].fontName = HANGEUL_FONT
    styles['Normal'].fontName = HANGEUL_FONT

    elements = []
    
    # --- 보고서 제목 ---
    elements.append(Paragraph("<b>은행 대출금리 신규취급액 기준 추이 분석 보고서</b>", styles['Title']))
    elements.append(Spacer(1, 18))
    
    # --- 원본 데이터 표 배치 ---
    elements.append(Paragraph("<b>1. 주요 대출 금리 추이 테이블 (신규 취급액 기준)</b>", styles['h2']))
    elements.append(Spacer(1, 6))

    # 데이터 준비 및 인덱스(월)를 컬럼으로 추가
    df_data = df_data.copy()
    df_data.index = df_data.index.strftime('%Y년 %m월')
    df_data.index.name = '기준월'
    
    # ReportLab Table 형식으로 변환
    data_list = [df_data.index.insert(0, df_data.index.name).tolist()] + [list(t) for t in df_data.itertuples(index=True, name=None)]
    
    table = Table(data_list, colWidths=[1.5*cm, 2.5*cm, 2.5*cm, 2.5*cm])
    
    # 표 스타일 설정 
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, -1), HANGEUL_FONT),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))
    elements.append(table)
    elements.append(Spacer(1, 24))

    # --- 시각화 그래프 배치 ---
    elements.append(Paragraph("<b>2. 대출금리 항목별 월별 추이 (선 그래프)</b>", styles['h2']))
    elements.append(Spacer(1, 6))

    # 이미지 삽입
    try:
        img = Image(image_path, width=500, height=300)
        elements.append(img)
    except Exception as e:
        elements.append(Paragraph(f"⚠️ 시각화 이미지 삽입 오류: 파일을 찾을 수 없습니다. ({e})", styles['Normal']))
        
    doc.build(elements)

    print("\n🎉 최종 PDF 보고서 생성이 완료되었습니다.")
    print(f"파일 위치: {pdf_filename}")

# ===============================================
# 메인 실행 함수 (에이전트 워크플로우 실행)
# ===============================================
from reportlab.lib.units import cm

def run_agent_workflow():
    """에이전트의 4단계와 5단계를 순차적으로 실행합니다."""
    
    # 4단계 시작: 데이터 정리 및 시각화
    df_loan_rates = load_and_preprocess_data(CSV_FILE_NAME)
    
    chart_dir = '02_Draw_Charts_Loan' # 이전 예제와 충돌 방지를 위해 폴더명 변경
    image_path = generate_line_chart(df_loan_rates, chart_dir)
    
    # 5단계 시작: PDF 보고서 완성
    report_dir = '03_Final_Report_Loan'
    create_pdf_report(df_loan_rates, image_path, report_dir)

if __name__ == "__main__":
    run_agent_workflow()


NameError: name 'content_fetcher' is not defined